In [3]:
import pandas as pd

In [14]:
books = pd.read_csv('../CSV Files/books.csv')
ratings = pd.read_csv('../CSV Files/ratings_merged.csv')
ratings = ratings.drop('Unnamed: 0', axis=1)
time = pd.read_csv('../CSV Files/time.csv')
time = time.drop('Unnamed: 0', axis=1)
### keep the first and the last 9 columns of book
stats = books.iloc[:, [0,-11, -10, -7, -6, -5, -4, -3]]

In [15]:
#Merge ratings and time on left time_id on right id
ratings = pd.merge(ratings, time, left_on='time_id', right_on='id').drop(columns=['id', 'flag'], axis=1)

### create dummy columns from the rating column
ratings = pd.get_dummies(ratings, columns=['rating'])

#Create a new dataframe with the following calculations: count of ratings, mean of ratings, and sum of ratings and sum of ratings 1-5
df_ratings = ratings.groupby(['book_id','year']).agg({'rating_1': 'sum', 'rating_2': 'sum', 'rating_3': 'sum', 'rating_4': 'sum', 'rating_5': 'sum'})

# Add total of votes per year
df_ratings['count'] = df_ratings['rating_1'] + df_ratings['rating_2'] + df_ratings['rating_3'] + df_ratings['rating_4'] + df_ratings['rating_5']

# Add sum of votes per year (weighted sum)
df_ratings['ratings_sum'] = (df_ratings['rating_1']*1 + df_ratings['rating_2']*2 + df_ratings['rating_3']*3 + df_ratings['rating_4']*4 + df_ratings['rating_5']*5)

# Add the average rating per year
df_ratings['avg_rating'] = (df_ratings['rating_1']*1 + df_ratings['rating_2']*2 + df_ratings['rating_3']*3 + df_ratings['rating_4']*4 + df_ratings['rating_5']*5) / df_ratings['count']

# Convert floats to int
df_ratings['rating_1'] = df_ratings['rating_1'].astype(int)
df_ratings['rating_2'] = df_ratings['rating_2'].astype(int)
df_ratings['rating_3'] = df_ratings['rating_3'].astype(int)
df_ratings['rating_4'] = df_ratings['rating_4'].astype(int)
df_ratings['rating_5'] = df_ratings['rating_5'].astype(int)
df_ratings['count'] = df_ratings['count'].astype(int)
df_ratings['ratings_sum'] = df_ratings['ratings_sum'].astype(int)

# Save to csv
df_ratings.to_csv('../CSV Files/statistics_ratings.csv')

In [16]:
statistics_ratings = pd.read_csv('../CSV Files/statistics_ratings.csv')

authorship = pd.read_csv('../CSV Files/authorship.csv')
# Merge the two dataframes on the 'book_id' column
merged_df = pd.merge(statistics_ratings, authorship, on='book_id', how='left')

# Extract the author_id for each book and add it as a new column in the ratings_merged dataframe
statistics_ratings['author_id'] = merged_df['author_id']

statistics_ratings.to_csv('../CSV Files/statistics_ratings.csv')

statistics_ratings

,book_id,year,rating_1,rating_2,rating_3,rating_4,rating_5,count,ratings_sum,avg_rating,author_id
0,1,2010,23,62,342,784,1162,2373,10119,4.264223,1
1,1,2011,18,63,295,768,1160,2304,9901,4.297309,1
2,1,2012,17,58,303,758,1121,2257,9679,4.288436,1
3,1,2013,26,65,286,749,1064,2190,9330,4.260274,1
4,1,2014,23,69,276,765,1144,2277,9769,4.290294,1
...,...,...,...,...,...,...,...,...,...,...,...
99990,10000,2015,0,0,1,10,5,16,68,4.250000,10000
99991,10000,2016,1,1,3,7,4,16,60,3.750000,10000
99992,10000,2017,0,0,3,7,6,16,67,4.187500,10000
99993,10000,2018,0,0,3,5,3,11,44,4.000000,10000
